# Entrenamiento del clasificador

In [ ]:
########################################################################
###################### Carga de módulos ################################
########################################################################

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from google.colab import drive, files
import zipfile
import os
import shutil
import matplotlib.pyplot as plt
import time

In [ ]:
########################################################################
###################### Montar Drive y descomprimir #####################
########################################################################

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo zip
ruta_zip = "/content/drive/MyDrive/animales.zip"
ruta_destino = "/content/dataset_animales"

# Descomprimir
with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
    zip_ref.extractall(ruta_destino)

In [ ]:
# === Rutas de entrada y salida ===
origen = "/content/dataset_animales/animales"
ruta_preentrenamiento = "/content/dataset_animales_dividido/preentrenamiento"
ruta_prueba = "/content/dataset_animales_dividido/prueba"

# Crear carpetas destino si no existen
os.makedirs(ruta_preentrenamiento, exist_ok=True)
os.makedirs(ruta_prueba, exist_ok=True)

# Recorrer cada clase
for clase in sorted(os.listdir(origen)):
    ruta_clase = os.path.join(origen, clase)
    if not os.path.isdir(ruta_clase):
        continue

    imagenes = os.listdir(ruta_clase)
    rutas_imagenes = [os.path.join(ruta_clase, img) for img in imagenes]

    # Dividir en 80% preentrenamiento y 20% prueba
    preentrenamiento, prueba = train_test_split(rutas_imagenes, test_size=0.2, random_state=42)

    # Crear carpetas destino por clase
    os.makedirs(os.path.join(ruta_preentrenamiento, clase), exist_ok=True)
    os.makedirs(os.path.join(ruta_prueba, clase), exist_ok=True)

    # Copiar archivos
    for img in preentrenamiento:
        shutil.copy(img, os.path.join(ruta_preentrenamiento, clase))

    for img in prueba:
        shutil.copy(img, os.path.join(ruta_prueba, clase))

print("✅ Separación completada:")
print("80% → preentrenamiento")
print("20% → prueba")


In [ ]:
########################################################################
###################### Parámetros generales ############################
########################################################################

img_height, img_width = 150, 150
batch_size = 32
epochs = 50
validation_split = 0.2

########################################################################
###################### Generadores de imágenes #########################
########################################################################

# Generador para preentrenamiento (con validación)
datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)

train_generator = datagen.flow_from_directory(
    ruta_preentrenamiento,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

ruta_txt = "nombres_clases.txt"
with open(ruta_txt, "w") as f:
    for nombre in train_generator.class_indices:
        f.write(nombre + "\n")

# Descargarlo a tu computadora (opcional)
from google.colab import files
files.download(ruta_txt)

val_generator = datagen.flow_from_directory(
    ruta_preentrenamiento,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Generador para prueba
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    ruta_prueba,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Clases
num_classes = len(train_generator.class_indices)
print("Clases detectadas:", train_generator.class_indices)

In [ ]:
########################################################################
###################### Modelo y compilación ############################
########################################################################

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

########################################################################
###################### Entrenamiento del modelo ########################
########################################################################

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

checkpoint = ModelCheckpoint("mejor_modelo.h5", monitor='val_accuracy',
                             save_best_only=True, verbose=1)

inicio = time.time()
historial = model.fit(train_generator,
                      validation_data=val_generator,
                      epochs=epochs,
                      callbacks=[early_stop, checkpoint])
fin = time.time()
duracion = fin - inicio
print(f"🕒 Entrenamiento completado en {duracion/60:.2f} minutos.")

In [ ]:
def graficar_metricas_entrenamiento(historial):
    epocas = range(1, len(historial.history['accuracy']) + 1)

    plt.figure(figsize=(12, 5))

    # Precisión
    plt.subplot(1, 2, 1)
    plt.plot(epocas, historial.history['accuracy'], label='Entrenamiento', marker='o')
    plt.plot(epocas, historial.history['val_accuracy'], label='Validación', marker='o')
    plt.title('Precisión por época')
    plt.xlabel('Época')
    plt.ylabel('Accuracy')
    plt.xticks(epocas)  # Mostrar todas las épocas enteras
    plt.legend()
    plt.grid(True)

    # Pérdida
    plt.subplot(1, 2, 2)
    plt.plot(epocas, historial.history['loss'], label='Entrenamiento', marker='o')
    plt.plot(epocas, historial.history['val_loss'], label='Validación', marker='o')
    plt.title('Pérdida por época')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.xticks(epocas)
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Llamada
graficar_metricas_entrenamiento(historial)


In [ ]:
########################################################################
###################### Evaluación en prueba ############################
########################################################################

mejor_val_acc = max(historial.history['val_accuracy'])
mejor_epoch = historial.history['val_accuracy'].index(mejor_val_acc) + 1
print(f"📈 Mejor precisión en validación: {mejor_val_acc:.4f} (época {mejor_epoch})")

In [ ]:
modelo_guardado = load_model("mejor_modelo.h5")
loss_test, acc_test = modelo_guardado.evaluate(test_generator, verbose=0)
print(f"🧪 Precisión en el conjunto de prueba: {acc_test:.4f}")
files.download("mejor_modelo.h5")

# Uso del clasificador

In [ ]:
########################################################################
#################### Carga de módulos ##################################
########################################################################

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from google.colab import drive, files
import zipfile
import os
import numpy as np
import pandas as pd

########################################################################
#################### Montar Drive y descomprimir #######################
########################################################################

drive.mount('/content/drive')

ruta_zip = "/content/drive/MyDrive/animales_desconocidos.zip"
ruta_destino = "/content/dataset_animales_desconocidos"

with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
    zip_ref.extractall(ruta_destino)

########################################################################
#################### Cargar modelo y nombres de clase ##################
########################################################################

modelo = load_model("mejor_modelo.h5")

with open("/content/nombres_clases.txt", "r") as f:
    clases = [line.strip() for line in f]

diccionario_indices = {i: nombre for i, nombre in enumerate(clases)}

########################################################################
#################### Predicción sobre nuevas imágenes ##################
########################################################################

ruta_imagenes = "/content/dataset_animales_desconocidos/animales_desconocidos"
tamaño = (150, 150)

resultados = []

for archivo in sorted(os.listdir(ruta_imagenes)):
    ruta_completa = os.path.join(ruta_imagenes, archivo)
    if not archivo.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img = image.load_img(ruta_completa, target_size=tamaño)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = modelo.predict(img_array, verbose=0)
    clase_predicha = np.argmax(pred)
    nombre_clase = diccionario_indices[clase_predicha]
    confianza = pred[0][clase_predicha]

    resultados.append({
        "archivo": archivo,
        "clase_predicha": nombre_clase,
        "confianza": f"{confianza:.2%}"
    })

########################################################################
#################### Guardar y mostrar resultados ######################
########################################################################

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

df_resultados.to_csv("predicciones_animales_desconocidos.csv", index=False)
files.download("predicciones_animales_desconocidos.csv")
